In [1]:
import json
import os
import collections
import numpy as np
import pandas as pd
import tensorflow as tf
import LAC
import happybase
import joblib
import matplotlib.pyplot as plt
import sklearn.model_selection

In [4]:
df_list = []
with open("/mnt1/zhaodachuan/data/predict_user_attribute20200911/raw_data/user_profile/aid_title_content") as f:
    for line in f:
        df_list.append(line.strip().split('\001'))
df = pd.DataFrame(df_list)

FileNotFoundError: [Errno 2] No such file or directory: '/mnt1/zhaodachuan/data/predict_user_attribute20200911/raw_data/user_profile/aid_title_content'

In [ ]:
df

In [ ]:
with open("/mnt1/zhaodachuan/data/predict_user_attribute20200911/raw_data/user_profile/content.jsonl","w") as f:
    for article in df_list:
        aid = article[0]
        content = article[-1]
        f.write(json.dumps({"aid":aid,"content":content},ensure_ascii=False) + "\n")

In [ ]:
with open("/mnt1/zhaodachuan/data/predict_user_attribute20200911/raw_data/user_profile/title.jsonl","w") as f:
    for article in df_list:
        aid = article[0]
        title = article[1]
        f.write(json.dumps({"aid":aid,"title":title},ensure_ascii=False) + "\n")

In [2]:
article_dict = {}
with open(r"D:\zhaodachuan\data\predict_user_attribute20200911\raw_data\user_profile\title_tokenize.jsonl",encoding="utf8") as f:
    for line in f:
        line_dict = json.loads(line.strip())
        aid = line_dict["aid"]
        article_dict[aid] = {}
        article_dict[aid]["title_tokenize"] = line_dict["title_tokenize"]
with open(r"D:\zhaodachuan\data\predict_user_attribute20200911\raw_data\user_profile\content_tokenize.jsonl",encoding="utf8") as f:
    for line in f:
        line_dict = json.loads(line.strip())
        aid = line_dict["aid"]
        if not aid in article_dict:
            print(aid)
            continue
        article_dict[aid]["content_tokenize"] = line_dict["content_tokenize"][:512]

In [3]:
#article_dict.keys()
article_dict['117568']

{'title_tokenize': ['医药',
  '电商',
  '瓶颈',
  '难',
  '解决',
  '，',
  '360',
  '网上',
  '卖',
  '药',
  '有',
  '谱',
  '吗',
  '？'],
 'content_tokenize': ['医药',
  '行业',
  '是',
  '最',
  '传统',
  '的',
  '行业',
  '之',
  '一',
  '，',
  '而',
  '医药',
  '电商',
  '则',
  '是',
  '一个',
  '专业性',
  '和',
  '安全性',
  '要求',
  '很高',
  '的',
  '细分',
  '市场',
  '，',
  '近年来',
  '医药',
  '电商',
  '也',
  '成',
  '了',
  '互联网',
  '巨头们',
  '瞄准',
  '的',
  '红利',
  '市场',
  '，',
  '各',
  '大',
  '巨头们',
  '摩拳擦掌',
  '准备',
  '拿下',
  '这',
  '一千亿',
  '市场',
  '。',
  '此前',
  '，',
  '阿里',
  '已经',
  '率先',
  '在',
  '天猫',
  '开通',
  '了',
  '医药',
  '类目',
  '，',
  '后来者',
  '腾讯',
  '、',
  '百度',
  '、',
  '京东',
  '等',
  '也',
  '都',
  '已',
  '形成',
  '了',
  '各自',
  '的',
  '布局',
  '和',
  '战略',
  '。',
  '除了',
  '这些',
  '互联网',
  '巨头们',
  '，',
  '其他',
  '一些',
  '中小',
  '玩家们',
  '也',
  '纷纷',
  '以',
  '不同',
  '的',
  '方式',
  '参与',
  '医药',
  '电商',
  '这',
  '一',
  '市场',
  '，',
  '如',
  '问',
  '药',
  '、',
  '快方',
  '送',
  '药',
  '、',
  '叮当',
  '送',
  '药',
  '等',

In [4]:
#预测集合
data_list = []
with open(r"D:\zhaodachuan\data\predict_user_attribute20200911\raw_data\user_profile\pre_sex_all_091116") as f:
    for line in f:
        data_list.append(line.strip().split("\001"))
df = pd.DataFrame(data_list,columns=["udid","age",*["c{}".format(i) for i in range(10)]])
del data_list
del df["age"]
df

,udid,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,1,380021,379528,380043,380126,379521,379529,379451,379526,378915,379489
1,6,381448,381295,381385,381284,381359,381552,381393,381540,381443,381309
2,8,379195,379107,379194,380776,378982,379529,379133,379489,379526,379004
3,38,366344,366247,366434,366043,366221,366113,366195,366379,366163,366182
4,44,378980,378933,379975,380021,377002,380043,376563,380126,378967,376989
...,...,...,...,...,...,...,...,...,...,...,...
261362,2902765,381483,381543,381474,381547,381539,381559,381552,381508,381542,381523
261363,2902766,381437,381474,381547,381313,362843,381542,381508,379878,380998,381523
261364,2902770,381453,381543,381474,381547,381537,381542,381535,381510,381540,381541
261365,2902775,381483,381385,381439,381559,381537,381552,381508,381510,381540,381523


In [5]:
text_tokenize_list_list = []
udid_list = []
for index in df.index:
    temp_list = []
    udid = df.loc[index,"udid"]
    for i in range(10):
        c = "c{}".format(i)
        if str(df.loc[index,c]):
            if str(df.loc[index,c]) in article_dict:
                temp_list.append([*article_dict[str(df.loc[index,c])]["title_tokenize"],"<PADDING>",*article_dict[str(df.loc[index,c])]["content_tokenize"]])
            else:
                print("aid不存在",index,c,df.loc[index,c])
                temp_list.append([])
        else:
            print(index)
            temp_list.append([])
    udid_list.append(udid)
    text_tokenize_list_list.append(temp_list)

aid不存在 45 c8 134
aid不存在 62 c3 130
aid不存在 75 c3 130
aid不存在 351 c8 134
aid不存在 424 c7 134
aid不存在 429 c5 130
aid不存在 609 c3 130
aid不存在 764 c4 130
aid不存在 881 c3 130
aid不存在 1233 c4 130
aid不存在 1544 c5 161
aid不存在 1695 c4 154
aid不存在 2131 c3 130
aid不存在 2163 c3 130
aid不存在 2818 c8 96
aid不存在 2859 c5 130
aid不存在 2877 c3 130
aid不存在 3096 c7 130
aid不存在 3492 c6 130
aid不存在 3730 c8 96
aid不存在 4812 c8 161
aid不存在 5074 c5 161
aid不存在 5525 c3 130
aid不存在 5629 c3 130
aid不存在 5808 c3 130
aid不存在 6014 c8 142
aid不存在 6779 c9 96
aid不存在 6834 c2 130
aid不存在 7343 c4 130
aid不存在 7383 c2 130
aid不存在 7416 c4 130
aid不存在 7467 c2 161
aid不存在 7877 c0 108
aid不存在 8664 c4 130
aid不存在 8758 c2 135
aid不存在 8758 c3 161
aid不存在 8848 c6 130
aid不存在 8853 c6 134
aid不存在 8900 c5 130
aid不存在 9026 c7 126
aid不存在 9260 c8 134
aid不存在 9509 c5 213
aid不存在 9606 c4 130
aid不存在 9649 c5 161
aid不存在 9835 c6 96
aid不存在 10461 c0 169
aid不存在 10478 c6 154
aid不存在 11371 c4 130
aid不存在 11847 c5 130
aid不存在 12263 c4 161
aid不存在 12442 c3 130
aid不存在 13693 c4 130
aid不存在 14043 c4 130
a

In [6]:
collections.Counter([len(v) for v in text_tokenize_list_list])

Counter({10: 261367})

In [7]:
len(udid_list)

261367

In [8]:
len(text_tokenize_list_list)

261367

In [9]:
# flag = "age"
# flag = "position"
flag = "trade"

In [10]:
APP_DIR = os.path.join(r"C:\Users\hnjyz\OneDrive\jupyter_lab\code\hx_recall_system\predict_user_attribute",flag)
with open(os.path.join(APP_DIR,"data","word2index.json")) as f:
    word2index_dict = json.load(f)

In [11]:
def trans2index(word2index_dict,word):
    """转换"""
    if word in word2index_dict:
        return word2index_dict[word]
    else:
        if "<UNK>" in word2index_dict:
            return word2index_dict["<UNK>"]
        else:
            raise ValueError("没有这个值，请检查")

In [12]:
word2index_dict["我"]

378

In [13]:
len(text_tokenize_list_list[0])

10

In [14]:
for text_tokenize_list in text_tokenize_list_list:
    print(text_tokenize_list)
    break

[['背叛', '乔布斯', '，', '库克', '做', '对', '了', '<PADDING>', '本文', '作者', '：', '张小旺', '，', '题', '图', '来自', '：', '视觉中国', '这', '是', '2011年', '，', '美国', '当地时间', '10月4日', '，', '旧金山草地艺术中心', '。', '稍加', '留意', '便', '不难', '发现', '，', '在', '这', '场', 'iPhone', ' ', '4S', '发布会', '上', '，', '多', '出来', '了', '一个', '空', '座位', '，', '上面', '标记', '着', '“', '保留', '”', '。', '那', '是', '乔布斯', '的', '保留', '席位', '，', '他', '因', '病', '没有', '出席', '这', '场', '发布会', '。', '就', '在', '第二天', '，', '这位', '可能', '是', '有史以来', '最', '伟大', '科技', '公司', '的', '创始人', '与世长辞', '。', '接替', '他', '成为', '苹果', '新', '掌门', '的', '，', '是', '公众', '并不', '熟悉', '的', '蒂姆·库克', '。', '乔布斯', '与', '病魔', '缠斗', '的', '几年间', '，', '库克', '先后', '两次', '短暂', '担任', '这家', '公司', '的', 'CEO', '，', '但', '并未', '引起', '人们', '的', '注意', '。', '就像', '那', '张', '保留', '的', '空', '座位', '一样', '，', '接下来', '很', '长', '的', '一段时间', '里', '，', '库克', '不得不', '面临', '这样', '一个', '事实', '：', '即便', '他', '已经', '成为', '这家', '公司', '的', '掌舵者', '，', '但', '人们', '还是', '忘不了', '乔布斯', '，', '对', '他', '这位', '新晋', 'CEO',

In [15]:
x_list = []
for text_tokenize_list in text_tokenize_list_list:
    x_list.append([[trans2index(word2index_dict,word) for word in word_list] for word_list in text_tokenize_list])
del text_tokenize_list_list

In [16]:
input_number = 10
x_max_length = 512

In [17]:
for i in range(len(x_list)):
    x_list[i] = tf.keras.preprocessing.sequence.pad_sequences(x_list[i],maxlen=x_max_length,dtype=np.int32,truncating="post", padding='post',value=0)

In [18]:
collections.Counter([len(v) for v in x_list])

Counter({10: 261367})

In [19]:
# strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
# with strategy.scope():
# 
model = tf.keras.models.load_model(os.path.join(r"C:\Users\hnjyz\OneDrive\jupyter_lab\code\hx_recall_system\predict_user_attribute",flag,"model_output"))

In [20]:
x_npa = np.array(x_list,dtype=np.int32)

In [21]:
res_list = model.predict({"input{}".format(i):x_npa[:,i] for i in range(input_number)},batch_size = 1024)

In [22]:
score_list = [np.argmax(v) for v in res_list]

In [23]:
collections.Counter(score_list)

Counter({3: 63595,
         1: 29455,
         4: 91471,
         6: 2205,
         9: 3487,
         0: 37838,
         7: 1506,
         2: 26189,
         8: 1389,
         5: 3340,
         10: 892})

In [24]:
with open(os.path.join(r"C:\Users\hnjyz\OneDrive\jupyter_lab\code\hx_recall_system\predict_user_attribute",flag,"data","tag2index.json"),encoding="utf8") as f:
    tag2index_dict = json.load(f)
index2tag_dict = {v:k for k,v in tag2index_dict.items()}

In [25]:
tag2index_dict

{'零售消费': 0,
 'IT/移动互联网': 1,
 '电商': 2,
 '金融': 3,
 '其他': 4,
 '游戏': 5,
 '制造业': 6,
 '汽车': 7,
 'IOT': 8,
 '房地产': 9,
 '医疗': 10}

In [26]:
{index2tag_dict[k]:v for k,v in collections.Counter(score_list).items()}

{'金融': 63595,
 'IT/移动互联网': 29455,
 '其他': 91471,
 '制造业': 2205,
 '房地产': 3487,
 '零售消费': 37838,
 '汽车': 1506,
 '电商': 26189,
 'IOT': 1389,
 '游戏': 3340,
 '医疗': 892}

In [27]:
res_df = pd.DataFrame()
res_df["uid"] = udid_list
res_df[flag] = score_list

In [28]:
res_df[flag] = res_df[flag].apply(lambda x : index2tag_dict[x])

In [29]:
res_df

,uid,trade
0,1,金融
1,6,IT/移动互联网
2,8,金融
3,38,金融
4,44,IT/移动互联网
...,...,...
261362,2902765,IT/移动互联网
261363,2902766,零售消费
261364,2902770,IT/移动互联网
261365,2902775,房地产


In [30]:
res_df.to_csv(os.path.join(r"C:\Users\hnjyz\OneDrive\jupyter_lab\code\hx_recall_system\predict_user_attribute",flag,"data","res_df.csv"),index=False)